<a href="https://colab.research.google.com/github/kbro4/Reliever-Value/blob/main/Individual_Relievers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
import warnings

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
# Datasets for each year
urls = {2013 : 'http://bbref.com/pi/shareit/zTQRZ',
        2014 : 'http://bbref.com/pi/shareit/BdM7N',
        2015 : 'http://bbref.com/pi/shareit/JTlEk',
        2016 : 'http://bbref.com/pi/shareit/d4JcY',
        2017 : 'http://bbref.com/pi/shareit/ORBJM',
        2018 : 'http://bbref.com/pi/shareit/ooBRY',
        2019 : 'http://bbref.com/pi/shareit/sbzjf',
        2021 : 'http://bbref.com/pi/shareit/hJVbP',
        2022 : 'http://bbref.com/pi/shareit/H8igd',
        2023 : 'http://bbref.com/pi/shareit/YeWWJ'
}

In [ ]:
# Filters and cleans dataframe to include only starters
def get_top_relievers(df):
  df = df.dropna()
  df = df[df['Rk'] != 'Rk']
  df = df.where(df['GS'].astype(float) / df['G'].astype(float) < 0.5)
  df = df.sort_values(by='WAR', ascending=False)
  df = df.dropna()
  return df

In [ ]:
# Checks to see if player is already in dataframe, if not appends new row for that player
def update_database(new_df, total_df, year):
  for name in new_df['Name']:
    if name in total_df['Name'].values:
      total_df.loc[total_df['Name'] == name, year] = new_df.loc[new_df['Name'] == name]['WAR'].iloc[0]
    else:
            # Add a new row with the person's name and specified value for '2013'
      new_row = {'Name': name, year: new_df.loc[new_df['Name'] == name]['WAR'].iloc[0]}
      new_row = pd.DataFrame({'Name': [name], year: [new_df.loc[new_df['Name'] == name]['WAR'].iloc[0]]})
      total_df = pd.concat([total_df, new_row], ignore_index=True)
  return total_df

In [ ]:
# Gets the top thirty relief pitchers by bWAR in a given year
def get_top_thirty(df):
  df = df.dropna()
  df = df[df['Rk'] != 'Rk']
  df = df.where(df['GS'].astype(float) / df['G'].astype(float) < 0.5)
  df = df.sort_values(by='WAR', ascending=False)
  df = df.dropna()
  df = df.head(30)
  return df

In [ ]:
# Gets how many of the top thirty relievers in a year are worth at least a given value one and two years later
def elite_consistency(df, total_df, year):
    sum_one_later = 0
    sum_two_later = 0
    num_rows = df.shape[0]  # Number of rows in the dataframe

    for index, row in df.iterrows():
        name = row['Name']

        # Check if the player's name exists in the total dataframe
        if name in total_df['Name'].values:
            player_data = total_df.loc[total_df['Name'] == name]
            # Check if data for year exists
            if year + 1 in player_data.columns:
              # Can hard code a value
                if player_data[year + 1].values[0] >= 0.7:
                  sum_one_later = sum_one_later + 1
            # Check if data for year exists
            if year + 2 in player_data.columns:
              # Can hard code a value
                if player_data[year + 2].values[0] >= 0.7:
                  sum_two_later = sum_two_later + 1

    return sum_one_later, sum_two_later


In [ ]:
# Creates a master dataframe of all relief pitchers during the time frame and value by year
total_df = pd.DataFrame(columns=['Name'])

for key in urls:
  df = pd.read_html(urls[key])
  df = df[0]

  new_df = get_top_relievers(df)
  total_df = update_database(new_df, total_df, key)
  time.sleep(2)


In [ ]:
total_df

,Name,2013,2014,2015,2016,2017,2018,2019,2021,2022,2023
0,Koji Uehara,3.5,1.7,1.2,0.7,0.2,NaN,NaN,NaN,NaN,NaN
1,Greg Holland,3.2,2.6,0.3,NaN,1.5,-0.1,0.1,0.0,-0.3,NaN
2,Joe Nathan,3.2,-0.3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Craig Kimbrel,3.1,2.5,1.3,0.8,3.6,2.1,-0.6,2.4,NaN,1.8
4,Rex Brothers*,3.1,-0.9,0.5,NaN,NaN,NaN,NaN,-0.3,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1096,Miles Mastrobuoni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1
1097,Josh Rojas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1
1098,Cole Sulser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1
1099,Lane Thomas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.1


In [15]:
for column in total_df:
  if column != 'Name':
    total_df[column] = total_df[column].astype(float)

In [16]:
# Gets average performance variation from previous year
war_change = total_df.iloc[:, 1:].diff(axis=1)
avg_change = war_change.abs().mean()
avg_change

2013         NaN
2014    1.007595
2015    0.930380
2016    0.925490
2017    1.025444
2018    1.079375
2019    0.890960
2021    0.884252
2022    1.053608
2023    0.955238
dtype: float64

In [17]:
# Gets average performance variation in two years
for column in total_df:
  if column != 'Name' and column + 2!= 2020 and column != 2022:
    change_two = total_df[column + 2] - total_df[column]
    avg_change_two = change_two.abs().mean()
    print(column, avg_change_two)
  elif column == 2022:
    break

2013 0.9621212121212122
2014 1.050413223140496
2015 1.0719696969696972
2016 1.0681159420289856
2017 1.0789855072463765
2019 0.8842519685039371
2021 1.0532467532467533


In [18]:
# Prints number of top thirty relievers will be worth a given value one and two years later
for key in urls:
  df = pd.read_html(urls[key])
  df = df[0]

  print(key, elite_consistency(get_top_thirty(df), total_df, key))
  time.sleep(2)

2013 (16, 13)
2014 (19, 17)
2015 (18, 15)
2016 (13, 11)
2017 (12, 10)
2018 (16, 0)
2019 (0, 11)
2021 (8, 7)
2022 (12, 0)
2023 (0, 0)
